## <center>Team Kullipojad<center>

For analysing the data, we used an introductory notebook provided by the kind **CaesarLupum**. 
The notebook is available here: https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction

Using CaesarLupum's notebook, we read the data in and reduced its memory usage with the following code:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 150)

py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import os,random, math, psutil, pickle    

In [ ]:
%%time
root = '../input/ashrae-energy-prediction/'
train_df = pd.read_csv(root + 'train.csv')
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')

weather_train_df = pd.read_csv(root + 'weather_train.csv')
test_df = pd.read_csv(root + 'test.csv')
weather_test_df = pd.read_csv(root + 'weather_test.csv')
building_meta_df = pd.read_csv(root + 'building_metadata.csv')
sample_submission = pd.read_csv(root + 'sample_submission.csv')

In [ ]:
print('Size of train_df data', train_df.shape)
print('Size of weather_train_df data', weather_train_df.shape)
print('Size of weather_test_df data', weather_test_df.shape)
print('Size of building_meta_df data', building_meta_df.shape)

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)

Merging, filling in NaNs etc

In [ ]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])
    
building_meta_df['primary_use'] = building_meta_df['primary_use'].astype('category')

In [ ]:
   
temp_df = train_df[['building_id']]
temp_df = temp_df.merge(building_meta_df, on=['building_id'], how='left')
del temp_df['building_id']
train_df = pd.concat([train_df, temp_df], axis=1)

temp_df = test_df[['building_id']]
temp_df = temp_df.merge(building_meta_df, on=['building_id'], how='left')

del temp_df['building_id']
test_df = pd.concat([test_df, temp_df], axis=1)
del temp_df, building_meta_df


In [ ]:
temp_df = train_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_train_df, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
train_df = pd.concat([train_df, temp_df], axis=1)

temp_df = test_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_test_df, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
test_df = pd.concat([test_df, temp_df], axis=1)

del temp_df, weather_train_df, weather_test_df

In [ ]:
train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')
   
del train_df, test_df
gc.collect()

In [ ]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [ ]:
train_df['age'] = train_df['year_built'].max() - train_df['year_built'] + 1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

In [ ]:

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 


In [ ]:
train_df['month_datetime'] = train_df['timestamp'].dt.month.astype(np.int8)
train_df['weekofyear_datetime'] = train_df['timestamp'].dt.weekofyear.astype(np.int8)
train_df['dayofyear_datetime'] = train_df['timestamp'].dt.dayofyear.astype(np.int16)
    
train_df['hour_datetime'] = train_df['timestamp'].dt.hour.astype(np.int8)  
train_df['day_week'] = train_df['timestamp'].dt.dayofweek.astype(np.int8)
train_df['day_month_datetime'] = train_df['timestamp'].dt.day.astype(np.int8)
train_df['week_month_datetime'] = train_df['timestamp'].dt.day/7
train_df['week_month_datetime'] = train_df['week_month_datetime'].apply(lambda x: math.ceil(x)).astype(np.int8)
    
train_df['year_built'] = train_df['year_built']-1900
train_df['square_feet'] = np.log(train_df['square_feet'])
    
test_df['month_datetime'] = test_df['timestamp'].dt.month.astype(np.int8)
test_df['weekofyear_datetime'] = test_df['timestamp'].dt.weekofyear.astype(np.int8)
test_df['dayofyear_datetime'] = test_df['timestamp'].dt.dayofyear.astype(np.int16)
    
test_df['hour_datetime'] = test_df['timestamp'].dt.hour.astype(np.int8)
test_df['day_week'] = test_df['timestamp'].dt.dayofweek.astype(np.int8)
test_df['day_month_datetime'] = test_df['timestamp'].dt.day.astype(np.int8)
test_df['week_month_datetime'] = test_df['timestamp'].dt.day/7
test_df['week_month_datetime'] = test_df['week_month_datetime'].apply(lambda x: math.ceil(x)).astype(np.int8)
    
test_df['year_built'] = test_df['year_built']-1900
test_df['square_feet'] = np.log(test_df['square_feet'])

### Our contribution to feature engineering

CaesarLupum's notebook provides an overview of what buildings were responsible for the max values in all four metrics. Because only 22 buildings made up the max values over dozens of days, we decided to eliminate these, as there may have been issues with the meters or some other explanation for why some values were unreasonably high, and we did not want these few outliers to cause changes in the results for the millions of other instances.

In [ ]:
# The IDs of the notorious 22 buildings, taken from the graphs in  CaesarLupum's notebook 
ids = [803,801,799,1088,993,794,1284,76,1258,1289,778,29,1156,29,60,50,1099,1197,1168,1148,1021,1221]

In [ ]:
# copy of the train and test dataframes 
train_copy = train_df.copy(deep=True)
test_copy = test_df.copy(deep=True)

Looking at the correlations between features, we found many that seemed to be worthless when determining the value of the meter, so we decided to drop these.

In [ ]:
to_drop = ['precip_depth_1_hr',
 'cloud_coverage',
 'wind_direction',
'primary_use' ,                            
'year_built',       
'weekofyear_datetime' ,
'month_datetime' ,   
'floor_count',          
'sea_level_pressure',   
'dew_temperature',       
'cloud_coverage',        
'day_week',              
'wind_direction',        
'day_month_datetime',    
'week_month_datetime',  
 ]

In [ ]:
# We drop the previous columns and remove the 22 buildings
train_copy = train_copy.drop(columns=to_drop,axis=1)
train_copy = train_copy[~train_copy.building_id.isin(ids)]


In [ ]:
# It was a headache to feed this value to the model so we dropped it and prayed that the day and the hour would be enough
train_copy = train_copy.drop(['timestamp'],axis=1)
test_copy = test_copy.drop(['timestamp'],axis=1)

We normalise some of the features with min-max normalisation, as they have very varying ranges and values.

In [ ]:
train_copy['square_feet']=(train_copy['square_feet']-train_copy['square_feet'].min())/(train_copy['square_feet'].max()-train_copy['square_feet'].min())
train_copy['air_temperature']=(train_copy['air_temperature']-train_copy['air_temperature'].min())/(train_copy['air_temperature'].max()-train_copy['air_temperature'].min())
train_copy['age']=(train_copy['age']-train_copy['age'].min())/(train_copy['age'].max()-train_copy['age'].min())
train_copy['wind_speed']=(train_copy['wind_speed']-train_copy['wind_speed'].min())/(train_copy['wind_speed'].max()-train_copy['wind_speed'].min())

Shuffle the train set and take out the value we want to predict.

In [ ]:
shuffled_train = train_copy.sample(frac=1)

In [ ]:
y_train=shuffled_train['meter_reading']
shuffled_train= shuffled_train.drop(['meter_reading'],axis=1)

Same goes for the test data, i.e drop useless columns and normalise.

In [ ]:
test_copy = test_copy.drop(columns=to_drop,axis=1)

In [ ]:
# save the row id separately
row_id = test_copy['row_id']
test_copy = test_copy.drop(['row_id'],axis=1)

In [ ]:
test_copy['square_feet']=(test_copy['square_feet']-test_copy['square_feet'].min())/(test_copy['square_feet'].max()-test_copy['square_feet'].min())
test_copy['air_temperature']=(test_copy['air_temperature']-test_copy['air_temperature'].min())/(test_copy['air_temperature'].max()-test_copy['air_temperature'].min())
test_copy['age']=(test_copy['age']-test_copy['age'].min())/(test_copy['age'].max()-test_copy['age'].min())
test_copy['wind_speed']=(test_copy['wind_speed']-test_copy['wind_speed'].min())/(test_copy['wind_speed'].max()-test_copy['wind_speed'].min())
test_copy['dayofyear_datetime']=(test_copy['dayofyear_datetime']-test_copy['dayofyear_datetime'].min())/(test_copy['dayofyear_datetime'].max()-test_copy['dayofyear_datetime'].min())
test_copy['hour_datetime']=(test_copy['hour_datetime']-test_copy['hour_datetime'].min())/(test_copy['hour_datetime'].max()-test_copy['hour_datetime'].min())

## Model 

We used an entirely new model we hadn't yet seen in neither the introductory nor the baseline notebook. We thought using some gradient boosting models would be a good idea, as the instructors went for the same idea and it seems to be a very popular thing in Kaggle competitions. We wanted to make use of something else besides LightGBM, so we chose XGBoost instead. This is a fairly common and powerful optimised and distributed gradient boosting library that is supposed to be efficient, fast and flexible as well as reasonably accurate. You can read more about XGBoost here: https://xgboost.readthedocs.io/en/latest/

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

We wanted to use the RMSLE function as the objective, but for some reason the version of XGBoost _pip_ automatically installed did not have this function included, so we made use of other ones. We chose a fairly general learning rate, and a max depth of 5 (the maximum is supposed to be 6). The documentation of XGBoost was massive and at times difficult to understand, or did not contain as much information as we would have liked, and we did not have much time to experiment with different parameters, so perhaps using a different learning rate, alpha, estimators etc would have yielded better results. Perhaps LightGBM would've been even better, but we wanted to try out an entirely new model instead of imitating the one chosen by the instructors. 

In [ ]:
xgboost_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
# Fitting
xgboost_reg.fit(shuffled_train,y_train)

In [ ]:
# Predicting
preds = xgboost_reg.predict(test_copy)

In [ ]:
# Writing the results into a file 
dt = pd.DataFrame(data=preds)
dt.to_csv('test_try_xgb.csv',  index=True)